In [1]:
import pandas as pd

In [2]:
#df = pd.read_csv('../data/raw/movies_dataset_2000_2023.csv')
df = pd.read_csv('movies_dataset_2000_2023.csv')

In [3]:
df.shape

(6095, 24)

In [4]:
df.columns

Index(['Unnamed: 0.1', 'Movie Title', 'Plot Summary', 'Cover Image', 'Year',
       'Source', 'Rated', 'omdb Year', 'omdb Title', 'Released', 'Runtime',
       'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country',
       'Awards', 'Poster', 'Ratings', 'imdbRating', 'BoxOffice', 'imdbID'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6095 entries, 0 to 6094
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  896 non-null    float64
 1   Movie Title   6095 non-null   object 
 2   Plot Summary  5556 non-null   object 
 3   Cover Image   5990 non-null   object 
 4   Year          6095 non-null   int64  
 5   Source        6095 non-null   object 
 6   Rated         841 non-null    object 
 7   omdb Year     871 non-null    object 
 8   omdb Title    871 non-null    object 
 9   Released      868 non-null    object 
 10  Runtime       861 non-null    object 
 11  Genre         870 non-null    object 
 12  Director      850 non-null    object 
 13  Writer        857 non-null    object 
 14  Actors        867 non-null    object 
 15  Plot          862 non-null    object 
 16  Language      866 non-null    object 
 17  Country       869 non-null    object 
 18  Awards        756 non-null  

In [6]:
!pip install nltk

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import string library for text preprocessing
import string

from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Convert text to lowercase
def convert_to_lower_case(text):
  return text.lower()


In [10]:
def remove_punctuation(text):
  # Create a translation table to remove punctuation
  translator = str.maketrans('', '', string.punctuation)

  # Remove punctuation using the translation table
  text_without_punct = text.translate(translator)

  return text_without_punct


In [11]:
def tokenize(text):
  # Tokenization
  return word_tokenize(text)

In [12]:
# Remove stopwords
def remove_stop_words(tokens):
  stop_words = set(stopwords.words('english'))
  return [word for word in tokens if word not in stop_words]

In [13]:
def lemmatize(tokens):
  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) if lemmatizer.lemmatize(word) is not None else word for word in tokens]



In [14]:
# Remove special characters and numbers
def remove_special_chars(tokens):
  return [re.sub('[^A-Za-z]+', '', word) for word in tokens]

In [15]:
# function to do preprocessing text
def preprocess_text(text):
  lower_text = convert_to_lower_case(text) # converting to lowercase
  removed_punctuation = remove_punctuation(lower_text) # removing punctuation
  tokens = tokenize(removed_punctuation) # tokenize words
  tokens = remove_stop_words(tokens) # removing stop words
  tokens = lemmatize(tokens) # lemmatize tokens
  tokens = remove_special_chars(tokens) # remove special characters from token

  preprocessed_text = ' '.join(tokens) # final preprocessed text
  return preprocessed_text

In [16]:
'''\
def preprocess_text(text):
  # Remove HTML tags and special characters
    clean_text = re.sub('<.*?>', '', text)
    clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)

    # Tokenization
    tokens = word_tokenize(clean_text)

    # Remove stop words and convert to lowercase
    stop_words = set(stopwords.words('english'))
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    return ' '.join(tokens)
    '''

"def preprocess_text(text):\n  # Remove HTML tags and special characters\n    clean_text = re.sub('<.*?>', '', text)\n    clean_text = re.sub(r'[^a-zA-Z\\s]', '', clean_text)\n\n    # Tokenization\n    tokens = word_tokenize(clean_text)\n\n    # Remove stop words and convert to lowercase\n    stop_words = set(stopwords.words('english'))\n    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]\n\n    return ' '.join(tokens)\n    "

In [17]:
df = df.dropna(subset=['Plot Summary'])

In [18]:
df['Plot Summary'].isna().sum()

0

In [19]:
# Apply function on text column
df['ProcessedPlot'] = df['Plot Summary'].apply(preprocess_text)

In [20]:
df.tail()

,Unnamed: 0.1,Movie Title,Plot Summary,Cover Image,Year,Source,Rated,omdb Year,omdb Title,Released,...,Plot,Language,Country,Awards,Poster,Ratings,imdbRating,BoxOffice,imdbID,ProcessedPlot
6090,NaN,Memory (2023 film),"Sylvia, a single mother, social worker, and re...",upload.wikimedia.org/wikipedia/en/thumb/4/4e/M...,2023,https://en.wikipedia.org//wiki/Memory_(2023_film),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sylvia single mother social worker recovering ...
6091,NaN,The Color Purple (2023 film),"In 1909 Georgia, teenager Celie Harris lives w...",upload.wikimedia.org/wikipedia/en/thumb/7/70/C...,2023,https://en.wikipedia.org//wiki/The_Color_Purpl...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,georgia teenager celie harris life sister net...
6092,NaN,The Boys in the Boat (film),The film's plot centres on the University of W...,upload.wikimedia.org/wikipedia/en/thumb/6/63/B...,2023,https://en.wikipedia.org//wiki/The_Boys_in_the...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,film plot centre university washington crew re...
6093,NaN,Ferrari (2023 film),"In the summer of 1957, Italian entrepreneur En...",upload.wikimedia.org/wikipedia/en/thumb/f/f6/F...,2023,https://en.wikipedia.org//wiki/Ferrari_(2023_f...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,summer italian entrepreneur enzo ferrari prep...
6094,NaN,Good Grief (film),"In London, Marc enjoys a Christmas party with ...",upload.wikimedia.org/wikipedia/en/thumb/1/14/G...,2023,https://en.wikipedia.org//wiki/Good_Grief_(film),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,london marc enjoys christmas party friend oliv...


In [21]:
# Save the preprocessed data
#preprocessed_path = '../data/processed/preprocessed_dataset.csv'
#df.to_csv(preprocessed_path, index=False)